# Unit of Work

The purpose of a unit of work is to group all our updates to the DB into a single 'unit' of updates. In relational DBs, this is often wrapped in a DB transaction, but we won't do that here....

In [1]:
from barin2.identity_map import IdentityMap
from barin2.instrumentation import get_state, Status, set_object_status

import pymongo
cli = pymongo.MongoClient('mongodb://mongo')
db = cli.barin2
imap = IdentityMap()

In [2]:
user_doc = db.users.find_one()
user_mdoc = imap.process(db.users, user_doc, Status.CLEAN)
user_mdoc

{'_id': ObjectId('615e1ff773b0c646ff5e2a57'),
 'name': 'Rick',
 'classes': ['Python', 'ML']}

In [3]:
get_state(user_mdoc)

<istate status=Status.CLEAN container=None>

# Thinking about state

Documents (and fields) can have various states beyond dirty/not dirty

- clean - memory matches db
- new - in memory, not in db
- dirty - in both, memory has changes
- deleted - in db, not in memory

In [4]:
get_state(user_mdoc['classes'])

<istate status=Status.CLEAN container={'_id': ObjectId('615e1ff773b0c646ff5e2a57'), 'name': 'Rick', 'classes': ['Python', 'ML']}>

Check our changes

In [5]:
user_mdoc['business'] = 'Arborian'
get_state(user_mdoc)

<istate status=Status.DIRTY container=None>

In [6]:
set_object_status(user_mdoc, Status.CLEAN)
get_state(user_mdoc)

<istate status=Status.CLEAN container=None>

In [7]:
user_mdoc.update(business='Arborian')
get_state(user_mdoc)

<istate status=Status.DIRTY container=None>

In [8]:
set_object_status(user_mdoc, Status.CLEAN)
user_mdoc['classes'].pop()
get_state(user_mdoc)

<istate status=Status.DIRTY container=None>